From the ECFS protocol we found that the best h-score is obtained for the top 10 ranked genes. Now we continue to analyze these genes.

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import matplotlib
import itertools
from networkx.algorithms import community
import community
import math
from pca import pca

In [2]:
def correlation(dataframe):
    """
    Function that inputs a dataframe of selected genes and 
        1. computes correlation values between the genes in the dataframe
        2. plots the correlation matrix (heatmap)
        3. plots the correlation histogram
        4. plots the KDE of the correlation histogram
        5. returns a dataframe of format 'node1' 'node2' 'correlation coefficient'
           stored in the 'links' variable   
    """
    import seaborn as sns
    corr = dataframe.corr(method='pearson')

    import itertools
    list2d = corr.values.tolist()
    merged = list(itertools.chain(*list2d))

    #Transform it in a links data frame (3 columns only):
    links = corr.stack().reset_index()
    links.columns = ['gene1', 'gene2','corr_value']
    return links

In [3]:
def filtered(df,threshold, thr_upper = 1):
    """Generates a data frame from a given dataframe and filters it removing the
    edges with a weight lower than a given threshold.
    Input:  * df: a dataframe containing 3 columns (source node, target node and
            weight) and a row for each edge
            * threshold: minimum weight that an edge has to have to not being filtered.
    Output: * df_filtered: filtered dataframe, has 3 columns, keyed 'gene1', 'gene2' and 'corr_value' and
    a row for each edge.
    """
    df_filtered=df.loc[ (df['corr_value'] >= threshold) & (df['corr_value'] <= thr_upper) & (df['gene1'] != df['gene2']) ]
    return df_filtered

def network(df):
    """Creates and returns a grap (G) from a dataframe (df) using NetworkX library.
    Input:  * df: dataframe with 3 columns, keyed 'gene1', 'gene2' and 'corr_value' and a row for each
            edge.
    Output: * G: Network graph
    """
    G=nx.from_pandas_edgelist(df, 'gene1', 'gene2', edge_attr = 'corr_value')
    return G

def network_plot(G):
    """Functions that from a given network G, creates the plot representation of this network
    Input:  * G: Network graph
    Output: * network plot
    
    """
    pos = nx.spring_layout(G, scale=4)  # double distance between all nodes
    fig, ax = plt.subplots(figsize=(20,20))

    nx.draw(G,pos, with_labels=True, node_color='skyblue', node_size=2000 ,edge_color='grey', alpha=0.8 ,linewidths=0.5, font_size=12)


In [4]:
def louvain(G):
    
    """Implements the Louvain community search algorithm in a graph (G) and returns a
    dictionary of the partitions.
    Input:  * G: a graph generated with NetworkX
    Output: * dic_nodes: a dictionary like {community_number:{'node_1 , ... , node_N'}}
    """

    # Starting with an initial partition of the graph and running the Louvain algorithm for Community Detection
    try:
        partition=community.best_partition(G, weight='corr_value') #--> it has to have a weight, otherwise it is just 36 communities (each node is one)    
   
        values=[partition.get(node) for node in G.nodes()]
        list_com=partition.values()
        # Creating a dictionary like {community_number:list_of_participants}
        dict_nodes={}

        # Populating the dictionary with items
        for each_item in partition.items():
            v= set()
            community_num=each_item[1]
            community_node=each_item[0]
            if community_num in dict_nodes:

                dict_nodes.get(community_num).add(community_node)

            else:
                #print('entered else')
                v.add(community_node)
                dict_nodes.update({community_num:v})

        # Creating a new graph to represent the communities created by the Louvain algorithm
        G_comm=nx.Graph()

        # Populating the data from the node dictionary created earlier
        G_comm.add_nodes_from(dict_nodes)

    
    except ValueError:
        #This is here because sometimes there is a bad node degree situation and the partition
        #and dict_nodes cannot be computed 
        partition = {'G7SNSen_mean': 0, 'Y3SNSen_mean': 1, 'G3SNSen_mean': 2, 'Y7MSen_mean': 3, 'Y3SSen_mean': 4, 'Y3M_mean': 5, 'G7SSen_mean': 6, 'G3SSen_mean': 7, 'Y7FNSen_mean': 8, 'G7M_mean': 9, 'G3MSen_mean': 10, 'Y3FNSen_mean': 11, 'G3FNSen_mean': 12, 'G7FNSen_mean': 13, 'Y3F_mean': 14, 'Y7MNSen_mean': 15, 'Y7SNSen_mean': 16, 'G7S_mean': 17, 'Y7F_mean': 18, 'Y3S_mean': 19, 'Y7SSen_mean': 20, 'G3M_mean': 21, 'G7MNSen_mean': 22, 'G3S_mean': 23, 'G3MNSen_mean': 24, 'G3FSen_mean': 25, 'Y7S_mean': 26, 'G7FSen_mean': 27, 'G7F_mean': 28, 'G7MSen_mean': 29, 'Y3MSen_mean': 30, 'Y3MNSen_mean': 31, 'Y7M_mean': 32, 'G3F_mean': 33, 'Y3FSen_mean': 34, 'Y7FSen_mean': 35}
        dict_nodes = {0: {'G7SNSen_mean'}, 1: {'Y3SNSen_mean'}, 2: {'G3SNSen_mean'}, 3: {'Y7MSen_mean'}, 4: {'Y3SSen_mean'}, 5: {'Y3M_mean'}, 6: {'G7SSen_mean'}, 7: {'G3SSen_mean'}, 8: {'Y7FNSen_mean'}, 9: {'G7M_mean'}, 10: {'G3MSen_mean'}, 11: {'Y3FNSen_mean'}, 12: {'G3FNSen_mean'}, 13: {'G7FNSen_mean'}, 14: {'Y3F_mean'}, 15: {'Y7MNSen_mean'}, 16: {'Y7SNSen_mean'}, 17: {'G7S_mean'}, 18: {'Y7F_mean'}, 19: {'Y3S_mean'}, 20: {'Y7SSen_mean'}, 21: {'G3M_mean'}, 22: {'G7MNSen_mean'}, 23: {'G3S_mean'}, 24: {'G3MNSen_mean'}, 25: {'G3FSen_mean'}, 26: {'Y7S_mean'}, 27: {'G7FSen_mean'}, 28: {'G7F_mean'}, 29: {'G7MSen_mean'}, 30: {'Y3MSen_mean'}, 31: {'Y3MNSen_mean'}, 32: {'Y7M_mean'}, 33: {'G3F_mean'}, 34: {'Y3FSen_mean'}, 35: {'Y7FSen_mean'}}

    return dict_nodes, partition

In [5]:
def plot_communities(G):
    
    dict_nodes,partition = louvain(G)
    pos = nx.spring_layout(G)   
    #list of correlation values
    durations = [i['corr_value'] for i in dict(G.edges).values()]
    plt.figure(figsize=(15, 10))
    plt.axis('off')
    nx.draw_networkx_nodes(G, pos, node_size=1800, cmap=plt.cm.RdYlBu, node_color=list(partition.values()))
    
    nx.draw_networkx_edges(G, pos, alpha=0.3, edge_color=durations, edge_cmap=plt.cm.Blues, width=2)
  
    nx.draw_networkx_labels(G,pos, font_size=7, font_color='black', font_weight='bold', alpha=1.0)
    plt.show()

In [6]:
def network_computing(df):
    """
    Function that inputs a dataframe, removes the geneID/symbol (first column of df) 
    and applies all the necessary functions to find the communities.
    """
    #removing first column from the dataframe (that column contains the symbol/geneID of the gene in the row)
    df = df.iloc[:,1:]
    
    #applying correlation function
    corr_df = correlation(df)
    
    #constructing the network
    G = network(corr_df)
    
    #finding communities
    dict_nodes, partition = louvain(G)
    
    return dict_nodes

In [7]:
def h_value(df, n):
    """
    Function that computes the h-score both cell-type-wise and community wise.
    In the case the number of communities is not 3, the h-score is set to 0.
    """
    
    dataframe = correlation(df) #where df will be a dataframe where there are k selected genes
    #print(dataframe)

    G = network(dataframe)
  
    #print(G)
    dict_nodes, partition = louvain(G) 
    #print(dict_nodes)
    sen = []
    nsen = []
    bas = []
    H_comms = []    
    
    for comm in dict_nodes.values():

        count_sen = 0
        count_nsen = 0
        count_bas = 0
        
        for string in comm:
            #counting sen cells in each community
            if "FSen" in string:
                count_sen +=1
            elif "SSen" in string:
                count_sen +=1
            elif "MSen" in string:
                count_sen+=1
                
            #counting nsen cells in each community
            elif "FNSen" in string:
                count_nsen +=1
            elif "SNSen" in string:
                count_nsen +=1
            elif "MNSen" in string:
                count_nsen+=1
                
            #counting bas cells in each community
            elif "F_" in string:
                count_bas +=1
            elif "S_" in string:
                count_bas +=1
            elif "M_" in string:
                count_bas +=1
            
        
        #computing entropy celltype-wise
        prob_sen_celltype = count_sen/12
        prob_nsen_celltype = count_nsen/12
        prob_bas_celltype = count_bas/12

        sen.append(prob_sen_celltype)
        nsen.append(prob_nsen_celltype)
        bas.append(prob_bas_celltype)
        
        #computing entropy community-wise
        prob_sen_comm = count_sen/len(comm)
        prob_nsen_comm = count_nsen/len(comm)
        prob_bas_comm = count_bas/len(comm)
        #ensure that no "prob" value is equal to 0, because log(0) is undefined
        if prob_sen_comm == 0:
            pS = 0
        else:
            pS = prob_sen_comm*np.log(prob_sen_comm)
        if prob_nsen_comm == 0:
            pN = 0
        else:
            pN = prob_nsen_comm*np.log(prob_nsen_comm)       
        if prob_bas_comm == 0:
            pB = 0
        else:
            pB = prob_bas_comm*np.log(prob_bas_comm) 
        
        H_comm = -1*(pS+pN+pB) 
    
        #computing entropy community-wise
        #storing entropy values for each community in a list named H_comms
        H_comms.append(H_comm)
    
    
    ncomms = len(dict_nodes)
    H_comm_total = np.sum(H_comms)
    
    #computing the entropy celltype-wise
    H_sen = -sum([p*np.log(p) if p !=0 else 0 for p in sen])
    H_nsen = -sum([p*np.log(p) if p !=0 else 0 for p in nsen])
    H_bas = -sum([p*np.log(p) if p !=0 else 0 for p in bas])
    H_celltype_total = H_sen + H_nsen + H_bas
    
    H_total = H_comm_total + H_celltype_total

    #calculating maximums
    H_celltype_max = -3*np.log(1/12)
    H_comm_max = -3*((1/3)*np.log(1/3))
 
    if ncomms == 3:
        h = (H_celltype_max-H_total)/H_celltype_max
    else:
        h = 0 
    return abs(round(h,4)), len(dict_nodes), dict_nodes 

In [16]:
def permutating_genes(df, n):
    """
    Function that perfoms permutations and chooses each time 9 out of the possible 10 genes.
    """
    h_list = []
    genes_list=[]
    n_of_com_list = []
    
    tuple_list = []
    
    gene_name = df.symbol.to_list()
    for list_of_selected_genes in itertools.combinations(gene_name, n):
        list_of_eliminated_genes = set(gene_name).difference(list_of_selected_genes)
        df_selected = df[df['symbol'].isin(list_of_selected_genes)]
        h, number_of_comm, dict_nodes = h_value(df_selected.iloc[:,1:37], 12)
                
        h_list.append(h)
        genes_list.append(len(list_of_selected_genes))
        n_of_com_list.append(number_of_comm)
        tuple_list.append((len(list_of_selected_genes),round(h,3), dict_nodes, number_of_comm, list_of_eliminated_genes))

    tuple_df= pd.DataFrame(tuple_list, columns=['num_of_genes', 'h_score', 'comm_structure', 'number_of_comm', 'non-selected genes'])

    return tuple_df

## Computations

### Top 10 genes

In [17]:
df_ranked= pd.read_csv("ranked_df_142.csv", sep="\t")
df_top10 = df_ranked.head(10).iloc[:,3:39]
df_top10

,G7F_mean,G7FNSen_mean,G7FSen_mean,G7M_mean,G7MNSen_mean,G7MSen_mean,G7S_mean,G7SNSen_mean,G7SSen_mean,Y7F_mean,...,G3SSen_mean,Y3F_mean,Y3FNSen_mean,Y3FSen_mean,Y3M_mean,Y3MNSen_mean,Y3MSen_mean,Y3SNSen_mean,Y3SSen_mean,Y3S_mean
0,1.108826,2.634362,7.292113,1.114405,0.000000,6.984152,3.854309,2.837667,4.025069,3.852787,...,14.321997,1.372702,3.080218,6.496544,13.002197,0.000000,41.515963,1.080518,14.369990,3.202665
1,1.264172,4.773241,5.320926,0.000000,0.000000,0.966213,0.000000,0.000000,0.000000,2.901175,...,3.904617,0.000000,0.371843,4.617450,1.449223,0.000000,3.798781,0.000000,1.655950,0.000000
2,53.471140,29.909353,7.166949,22.672213,9.201572,4.214950,15.337308,3.235999,0.604172,26.068653,...,2.297985,31.290217,5.483048,6.133262,24.466720,5.408345,2.762375,1.169546,1.320244,34.169720
3,10.729073,14.389570,8.495852,10.363422,10.460677,8.948537,16.355583,13.633727,11.497847,7.569894,...,9.674773,9.580695,9.364737,8.473849,10.821351,10.569788,6.111948,12.020820,9.713858,12.113660
4,16.972573,14.361967,8.451580,11.245947,11.973417,9.386507,19.330207,20.186983,13.063754,11.451037,...,15.373570,10.445293,8.347679,6.340688,15.855947,25.132637,6.403587,14.182250,12.527593,21.817175
5,0.000000,0.045431,0.214271,0.000000,0.120286,0.685069,0.000000,0.000000,0.889377,0.000000,...,0.438844,0.000000,0.019417,0.333257,0.000000,0.000000,0.270568,0.000000,0.141918,0.000000
6,18.424960,13.046955,8.329151,12.811153,9.040402,7.524995,13.682937,11.960306,10.526299,12.648130,...,6.232045,17.028977,8.066060,7.457584,9.994184,8.460669,5.803649,7.741266,7.172253,16.954535
7,0.000000,1.228474,3.130106,0.000000,0.000000,2.969120,0.000000,0.000000,0.000000,0.648264,...,3.175385,0.845789,5.560121,8.081729,1.805436,0.000000,10.820645,0.000000,1.259414,0.031789
8,0.000000,0.000000,0.310584,0.000000,0.094287,0.000000,0.000000,0.000000,0.026277,0.000000,...,0.182541,0.000000,0.000000,0.257265,0.000000,0.000000,0.000000,0.000000,0.169502,0.000000
9,0.549420,0.000000,1.651297,0.000000,1.350701,3.531358,7.777188,0.121548,11.251232,2.074818,...,5.922007,0.000000,0.847302,1.592959,0.260174,0.000000,4.280093,0.740522,3.647594,7.698993


### Assesing the importance of individual genes present in top 10

In [18]:
df_top10genes = df_ranked.head(10).iloc[:,2:39]
df_top10genes

,symbol,G7F_mean,G7FNSen_mean,G7FSen_mean,G7M_mean,G7MNSen_mean,G7MSen_mean,G7S_mean,G7SNSen_mean,G7SSen_mean,...,G3SSen_mean,Y3F_mean,Y3FNSen_mean,Y3FSen_mean,Y3M_mean,Y3MNSen_mean,Y3MSen_mean,Y3SNSen_mean,Y3SSen_mean,Y3S_mean
0,Fabp3,1.108826,2.634362,7.292113,1.114405,0.000000,6.984152,3.854309,2.837667,4.025069,...,14.321997,1.372702,3.080218,6.496544,13.002197,0.000000,41.515963,1.080518,14.369990,3.202665
1,Gm38050,1.264172,4.773241,5.320926,0.000000,0.000000,0.966213,0.000000,0.000000,0.000000,...,3.904617,0.000000,0.371843,4.617450,1.449223,0.000000,3.798781,0.000000,1.655950,0.000000
2,Lncpint,53.471140,29.909353,7.166949,22.672213,9.201572,4.214950,15.337308,3.235999,0.604172,...,2.297985,31.290217,5.483048,6.133262,24.466720,5.408345,2.762375,1.169546,1.320244,34.169720
3,Pcnp,10.729073,14.389570,8.495852,10.363422,10.460677,8.948537,16.355583,13.633727,11.497847,...,9.674773,9.580695,9.364737,8.473849,10.821351,10.569788,6.111948,12.020820,9.713858,12.113660
4,Luc7l3,16.972573,14.361967,8.451580,11.245947,11.973417,9.386507,19.330207,20.186983,13.063754,...,15.373570,10.445293,8.347679,6.340688,15.855947,25.132637,6.403587,14.182250,12.527593,21.817175
5,Micalcl,0.000000,0.045431,0.214271,0.000000,0.120286,0.685069,0.000000,0.000000,0.889377,...,0.438844,0.000000,0.019417,0.333257,0.000000,0.000000,0.270568,0.000000,0.141918,0.000000
6,Ash1l,18.424960,13.046955,8.329151,12.811153,9.040402,7.524995,13.682937,11.960306,10.526299,...,6.232045,17.028977,8.066060,7.457584,9.994184,8.460669,5.803649,7.741266,7.172253,16.954535
7,AI427809,0.000000,1.228474,3.130106,0.000000,0.000000,2.969120,0.000000,0.000000,0.000000,...,3.175385,0.845789,5.560121,8.081729,1.805436,0.000000,10.820645,0.000000,1.259414,0.031789
8,Cdhr4,0.000000,0.000000,0.310584,0.000000,0.094287,0.000000,0.000000,0.000000,0.026277,...,0.182541,0.000000,0.000000,0.257265,0.000000,0.000000,0.000000,0.000000,0.169502,0.000000
9,Cd59a,0.549420,0.000000,1.651297,0.000000,1.350701,3.531358,7.777188,0.121548,11.251232,...,5.922007,0.000000,0.847302,1.592959,0.260174,0.000000,4.280093,0.740522,3.647594,7.698993


In [19]:
tuple_perm_df = permutating_genes(df_top10genes,9)

In [20]:
tuple_perm_df

,num_of_genes,h_score,comm_structure,number_of_comm,non-selected genes
0,9,0.654,"{0: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",3,{Cd59a}
1,9,0.737,"{0: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",3,{Cdhr4}
2,9,0.000,"{0: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",4,{AI427809}
3,9,0.737,"{1: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",3,{Ash1l}
4,9,0.737,"{0: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",3,{Micalcl}
5,9,0.000,"{0: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",4,{Luc7l3}
6,9,0.000,"{3: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",4,{Pcnp}
7,9,0.370,"{2: {'G7M_mean', 'Y3MNSen_mean', 'Y3FNSen_mean...",3,{Lncpint}
8,9,0.737,"{0: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",3,{Gm38050}
9,9,0.605,"{0: {'G7M_mean', 'G7FNSen_mean', 'G7F_mean', '...",3,{Fabp3}
